In [84]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [85]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\matth\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [86]:
url = "https://www.linkedin.com/jobs/search?keywords=Data%20Scientist&location=Kansas%2C%20City%2C%20Missouri%2C%20United%2C%20States&geoId=106142749&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0"

In [87]:
browser.visit(url)
html = browser.html
jobs_soup = soup(html, 'html.parser')



In [88]:
no_jobs = jobs_soup.find('span', class_='results-context-header__job-count').text
no_jobs_int = int(no_jobs.replace('+', '').replace(',', ''))

In [89]:
print(no_jobs_int)

802


In [90]:
i = 2
while i <= int(no_jobs_int/25)+1:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    try:
        browser.find_by_xpath('/html/body/div[1]/div/main/section[2]/button').click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)
    print(i)            

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [61]:
html = browser.html
jobs_soup = soup(html, 'html.parser')

In [62]:
job_list = jobs_soup.find('ul', class_='jobs-search__results-list')


In [63]:
jobs = job_list.find_all('li')

In [68]:
print(jobs)

[<li>
<div class="base-card base-card--link base-search-card base-search-card--link job-search-card job-search-card--active" data-column="1" data-entity-urn="urn:li:jobPosting:2836799662" data-row="1" data-search-id="jxHkQTgFrL1hhlq58C3ZzA==" data-tracking-id="8cZVwWyUvQF0t34RLMiPhw==">
<a class="base-card__full-link" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-will-navigate="" href="https://www.linkedin.com/jobs/view/data-scientist-at-hallmark-aviation-2836799662?refId=jxHkQTgFrL1hhlq58C3ZzA%3D%3D&amp;trackingId=8cZVwWyUvQF0t34RLMiPhw%3D%3D&amp;position=1&amp;pageNum=0&amp;trk=public_jobs_jserp-result_search-card">
<span class="screen-reader-text">
            
        
        Data Scientist
      
      
          </span>
</a>
<div class="search-entity-media">
<img alt="" class="artdeco-entity-image artdeco-entity-image--square-4 lazy-loaded" data-ghost-classes="artdeco-entity-image--ghost" data-ghost-url="https://s

In [71]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []

for job in jobs:
    try:
        job_id0 = job.div['data-row']
        job_id.append(job_id0)
        print(job_id0)

        job_title0 = job.find('h3', class_='base-search-card__title').get_text()
        job_title.append(job_title0.replace('\n', '').strip())
        print(job_title0)

        company_name0 = job.find('h4', class_='base-search-card__subtitle').get_text()
        company_name.append(company_name0.replace('\n', '').strip())

        location0 = job.find('span', class_='job-search-card__location').get_text()
        location.append(location0.replace('\n', '').strip())
        
        date0 = job.time['datetime']
        date.append(date0.replace('\n', '').strip())
        
        job_link0 = job.find('a', class_='base-card__full-link')['href']
        job_link.append(job_link0.replace('\n', '').strip())
    except:
        pass


1

            
        Data Scientist
      
          
2

            
        Associate Data Scientist
      
          
3

            
        Data Scientist
      
          
4

            
        Data Scientist
      
          
5

            
        Data Scientist
      
          
6

            
        Data Scientist
      
          
7

            
        Data Scientist
      
          
8

            
        Data Scientist - Bot Management
      
          
9

            
        Data Scientist
      
          
10

            
        Data Scientist, Capacity Engineering - Location Flexible
      
          
11

            
        Data Scientist
      
          
12

            
        Data Scientist (Remote)
      
          
13

            
        Data Scientist
      
          
14

            
        Data Scientist
      
          
15

            
        Data Scientist II
      
          
16

            
        Associate Data Scientist
      
 

In [72]:
date

['2021-12-09',
 '2021-12-12',
 '2021-12-09',
 '2021-11-26',
 '2021-11-24',
 '2021-10-31',
 '2021-12-11',
 '2021-12-10',
 '2021-11-23',
 '2021-10-31',
 '2021-11-07',
 '2021-11-23',
 '2021-12-02',
 '2021-10-31',
 '2021-12-14',
 '2021-12-10',
 '2021-10-31',
 '2021-12-04',
 '2021-10-31',
 '2021-11-01',
 '2021-12-02',
 '2021-11-13',
 '2021-12-02',
 '2021-10-27',
 '2021-12-04',
 '2021-12-11',
 '2021-12-14',
 '2021-11-17',
 '2021-10-29',
 '2021-12-02',
 '2021-11-09',
 '2021-11-12',
 '2021-10-02',
 '2021-12-06',
 '2021-11-30',
 '2021-11-02',
 '2021-12-14',
 '2021-12-09',
 '2021-11-15',
 '2021-10-31',
 '2021-11-05',
 '2021-10-05',
 '2021-11-01',
 '2021-11-01',
 '2021-12-14',
 '2021-11-12',
 '2021-11-05',
 '2021-10-31',
 '2021-10-20',
 '2021-10-27',
 '2021-12-07',
 '2021-12-05',
 '2021-12-04',
 '2021-11-17',
 '2021-11-28',
 '2021-12-06',
 '2021-11-19',
 '2021-12-01',
 '2021-11-19',
 '2021-12-13',
 '2021-10-22',
 '2021-12-13',
 '2021-11-16',
 '2021-10-13',
 '2021-12-03',
 '2021-12-12',
 '2021-12-

In [36]:
job_id

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75']

In [21]:
company_name

['IBM',
 'TPD®',
 'Cardinal Intellectual Property',
 'nugget.ai',
 'nugget.ai',
 'Equifax Canada Co.',
 'Ample Insight',
 'Deloitte',
 'FreshBooks',
 'Equifax Canada Co.',
 'Ample Insight',
 'Tiger Analytics',
 'Deloitte',
 'Daisy Intelligence Corporation',
 'LotusFlare',
 'Equifax Canada Co.',
 'Wish',
 'Validere',
 'Deloitte',
 'DoorDash',
 'Equifax Canada Co.',
 'Coalition, Inc.',
 'Clarivate',
 'Yelp',
 'Daily Blends',
 'Deloitte',
 'Tiger Analytics',
 'Aviva Canada',
 'TextNow',
 'Moneris',
 'Amazon',
 'Brunel',
 'MonetizeMore',
 'Priceline',
 'Onex',
 'VendorPM',
 'Tiger Analytics',
 'Bolt',
 'OANDA',
 'Ceridian',
 'Electronic Arts (EA)',
 'Ceridian',
 'FGF Brands',
 'Circle',
 'Ceridian',
 'HomeStars',
 'Shopify',
 'Ceridian',
 'Ceridian',
 'Ceridian',
 'Ceridian',
 'Uplift Inc.',
 'Ceridian',
 'Shift Technology',
 'Ceridian',
 'Zynga',
 'Viral Nation',
 'Interac Corp.',
 'Aviva Canada',
 'Clarivate',
 '[24]7.ai',
 'Coalition, Inc.',
 'Georgian',
 'Deep Genomics',
 'Ontario Gove

In [ ]:
job_title

['Associate Data Scientist (Entry Level): Toronto/Calgary',
 'Data Scientist',
 'Data Scientist/ Machine Learning Engineer - Remote',
 'Data Scientist',
 'Data Scientist',
 'Data Scientist',
 'Data Scientist (remote)',
 'Data Scientist',
 'Data Scientist',
 'Data Scientist',
 'Data Scientist - Data Analytics and Infrastructure (remote)',
 'Data Scientist (Remote)',
 'Data Scientist, Omnia AI - New Grad 2022 - National',
 'Data Scientist',
 'Data Scientist (Product Analytics)',
 'Data Scientist - Custom Projects',
 'Data Scientist',
 'Data Scientist',
 'Data Scientist, Omnia AI',
 'Data Scientist, International Analytics',
 'Data Scientist',
 'Data Scientist - Marketing Analytics',
 'Data Scientist (Healthcare Data & Solutions)',
 'Data Scientist (Remote)',
 'Data Scientist/Machine Learning Engineer - All Levels',
 'Data Scientist - DTTL Global Technology Solutions - Summer 2022 Co-op/Intern - Toronto',
 'Data Scientist (Canada)',
 'New Grad Software Engineer / Machine Learning Engineer

In [73]:
job_df = pd.DataFrame({'CompanyName': company_name, 'JobTitle': job_title, 'Location': location, 'DateListed': date, 'JobLink': job_link}, index=job_id)

job_df

,CompanyName,JobTitle,Location,DateListed,JobLink
1,Hallmark Aviation,Data Scientist,"Kansas City, MO",2021-12-09,https://www.linkedin.com/jobs/view/data-scient...
2,Cox Communications,Associate Data Scientist,"Kansas City, MO",2021-12-12,https://www.linkedin.com/jobs/view/associate-d...
3,Hallmark Cards,Data Scientist,"Kansas City, KS",2021-12-09,https://www.linkedin.com/jobs/view/data-scient...
4,Kansas City Symphony,Data Scientist,"Kansas City, MO",2021-11-26,https://www.linkedin.com/jobs/view/data-scient...
5,Leverage Lab,Data Scientist,Kansas City Metropolitan Area,2021-11-24,https://www.linkedin.com/jobs/view/data-scient...
...,...,...,...,...,...
608,WellSky,Sr. UX Researcher,"Overland Park, KS",2021-12-10,https://www.linkedin.com/jobs/view/sr-ux-resea...
609,Ad Hoc LLC,Senior UX Researcher II,"Kansas City, MO",2021-12-03,https://www.linkedin.com/jobs/view/senior-ux-r...
610,Ad Hoc LLC,Senior UX Researcher II,"Overland Park, KS",2021-12-03,https://www.linkedin.com/jobs/view/senior-ux-r...
611,Cardinal Health,"Lead, UX Researcher","Kansas City, MO",2021-11-25,https://www.linkedin.com/jobs/view/lead-ux-res...


In [38]:
job_df

,CompanyName,JobTitle
1,IBM,Associate Data Scientist (Entry Level): Toront...
2,TPD®,Data Scientist
3,Cardinal Intellectual Property,Data Scientist/ Machine Learning Engineer - Re...
4,nugget.ai,Data Scientist
5,nugget.ai,Data Scientist
...,...,...
71,Sharethrough,Data Scientist
72,Rubicon HR Canada,Toronto - R&D Data Scientist
73,Spin Master,Data Scientist
74,Drop,"Data Scientist, Algorithms"


In [83]:
browser.quit()